## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 200

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0005

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 160]

pretrained_model       = './cifar10_wrn_28_10_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## WideResNet-28-10 model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarWrnConfig(k = 10, num_layers = 28, cifar = 10)
model = model_gen.GetCifarWrn()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 2.041889, Test Acc: 38.69 %
2 epoch end, loss: 1.371702, Test Acc: 55.21 %
3 epoch end, loss: 1.005683, Test Acc: 65.73 %
4 epoch end, loss: 0.762805, Test Acc: 72.03 %
5 epoch end, loss: 0.629238, Test Acc: 77.65 %
6 epoch end, loss: 0.551587, Test Acc: 77.08 %
7 epoch end, loss: 0.503028, Test Acc: 69.35 %
8 epoch end, loss: 0.462662, Test Acc: 82.30 %
9 epoch end, loss: 0.438482, Test Acc: 79.35 %
10 epoch end, loss: 0.417109, Test Acc: 81.07 %
11 epoch end, loss: 0.399895, Test Acc: 82.81 %
12 epoch end, loss: 0.382185, Test Acc: 82.51 %
13 epoch end, loss: 0.370292, Test Acc: 80.93 %
14 epoch end, loss: 0.361261, Test Acc: 83.72 %
15 epoch end, loss: 0.346596, Test Acc: 79.16 %
16 epoch end, loss: 0.340734, Test Acc: 84.95 %
17 epoch end, loss: 0.332941, Test Acc: 86.10 %
18 epoch end, loss: 0.321516, Test Acc: 78.14 %
19 epoch end, loss: 0.317103, Test Acc: 84.74 %
20 epoch end, loss: 0.315574, Test Acc: 82.65 %
21 epoch end, loss: 0.307587, Test Acc: 82.54 %
2

171 epoch end, loss: 0.001671, Test Acc: 95.60 %
172 epoch end, loss: 0.001784, Test Acc: 95.70 %
173 epoch end, loss: 0.001701, Test Acc: 95.64 %
174 epoch end, loss: 0.001690, Test Acc: 95.57 %
175 epoch end, loss: 0.001648, Test Acc: 95.58 %
176 epoch end, loss: 0.001744, Test Acc: 95.58 %
177 epoch end, loss: 0.001765, Test Acc: 95.66 %
178 epoch end, loss: 0.001748, Test Acc: 95.64 %
179 epoch end, loss: 0.001761, Test Acc: 95.68 %
180 epoch end, loss: 0.001680, Test Acc: 95.66 %
181 epoch end, loss: 0.001690, Test Acc: 95.67 %
182 epoch end, loss: 0.001762, Test Acc: 95.57 %
183 epoch end, loss: 0.001660, Test Acc: 95.70 %
184 epoch end, loss: 0.001692, Test Acc: 95.67 %
185 epoch end, loss: 0.001649, Test Acc: 95.60 %
186 epoch end, loss: 0.001682, Test Acc: 95.62 %
187 epoch end, loss: 0.001648, Test Acc: 95.62 %
188 epoch end, loss: 0.001697, Test Acc: 95.58 %
189 epoch end, loss: 0.001646, Test Acc: 95.64 %
190 epoch end, loss: 0.001635, Test Acc: 95.53 %
191 epoch end, loss:

In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 95.67 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)